# Instructions

The assignment consists of three tasks:

- Run the T-test for the means of two independent samples underlying the statement "IRE binding activity was significantly reduced in failing hearts" (originally published by Haddad et al. in https://doi.org/10.1093/eurheartj/ehw333) using the following example data.

| non-failing heart (NF) | failing heart (F) |
| ---------------------- | ----------------- |
| 95 | 50 |
| 103 | 35 |
| 99 | 21 | 
| &nbsp; | 15 | 
| &nbsp; | 7 | 
| &nbsp; | 40 |

- Describe the statistical hypothesis test in machine readable form following the [statistical methods ontology concept for "two sample t-test with unequal variance"](https://www.ebi.ac.uk/ols/ontologies/stato/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FSTATO_0000304) using Semantic Web technologies, namely the Resource Description Framework (RDF)
- Process the resulting machine readable description using Semantic Web technologies, namely the SPARQL Protocol and RDF Query Language.

Please return the assignment with all outputs visible (i.e., do not clear the outputs).

Good luck!

In [1]:
!pip install rdflib pandas scipy numpy

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
# Import all required libraries (some are missing)
import pandas as pd
import numpy as np
import itertools
from scipy.stats import ttest_ind
from rdflib import Graph, URIRef
from rdflib.namespace import RDF
from rdflib.namespace import XSD
from rdflib import BNode, Literal

In [3]:
# Run the T-test for the means of two independent samples using the example data

labels = ["non-failing heart (NF)", "failing heart (F)"]
data = [[95, 103, 99], [50, 35, 21, 15, 7, 40]]
df = pd.DataFrame((_ for _ in itertools.zip_longest(*data)), columns=labels) 

_ , pvalue = ttest_ind(df.dropna()['non-failing heart (NF)'],df['failing heart (F)'])

# Print the p-value
print(pvalue)

0.00018016897790311737


In [6]:
# Describe the statistical hypothesis test in machine readable form

# First, we initialize an RDF Graph and bind some prefixes
g = Graph()
g.bind('obo', 'http://purl.obolibrary.org/obo/')
g.bind('ex', 'http://example.org/')

# Next, we define some needed vocabulary
obo = dict()
obo['two sample t-test with unequal variance'] = URIRef('http://purl.obolibrary.org/obo/STATO_0000304')
obo['has specified input'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000293')
obo['has specified output'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000299')
obo['p-value'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000175')
obo['has value specification'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001938')
obo['scalar value specification'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001931')
obo['has specified numeric value'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001937')
obo['iron-responsive element binding'] = URIRef('http://purl.obolibrary.org/obo/GO_0030350')

# Now, populate the graph with statements
# As an example, the following statement types the resource (blank node) 'n1' as 'two sample t-test with unequal variance'
n1 = BNode()
g.add((n1, RDF.type, obo['two sample t-test with unequal variance']))

# Now, state that the t-test has iron-responsive element binding as a specified input (one statement to complete)
n2 = BNode()
g.add((n1, obo['has specified input'], n2))
g.add((n2, RDF.type, obo['iron-responsive element binding']))

# Next, state that the t-test has the p-value computed above as a specified output (two statements to complete)
n3 = BNode() 
g.add((n1, obo['has specified output'], n3))
g.add((n3, RDF.type, obo['p-value']))

# The literal value of the p-value is represented as a scalar value specification with a specified numeric value (three statements to complete)
n4 = BNode() 
g.add((n3, obo['has value specification'], n4))
g.add((n4, RDF.type, obo['scalar value specification']))
# Type the literal as double
g.add((n3, obo['has specified numeric value'], Literal(pvalue, datatype=XSD.double)))

# Finally, serialize and print the graph in RDF/XML format
print (g.serialize(format='xml'))

b'<?xml version="1.0" encoding="UTF-8"?>\n<rdf:RDF\n   xmlns:obo="http://purl.obolibrary.org/obo/"\n   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n>\n  <rdf:Description rdf:nodeID="Nea0cd147cecd4958949b81b4a53d973d">\n    <obo:OBI_0000299 rdf:nodeID="N1ed56fead8194652a23d2d5a1c54e9b9"/>\n    <obo:OBI_0000293 rdf:nodeID="Nb3b56cb18d684ab7af65c46a42ede574"/>\n    <rdf:type rdf:resource="http://purl.obolibrary.org/obo/STATO_0000304"/>\n  </rdf:Description>\n  <rdf:Description rdf:nodeID="Nb3b56cb18d684ab7af65c46a42ede574">\n    <rdf:type rdf:resource="http://purl.obolibrary.org/obo/GO_0030350"/>\n  </rdf:Description>\n  <rdf:Description rdf:nodeID="N1ed56fead8194652a23d2d5a1c54e9b9">\n    <rdf:type rdf:resource="http://purl.obolibrary.org/obo/OBI_0000175"/>\n    <obo:OBI_0001938 rdf:nodeID="Nda297d0d0ee4404ca8ea326f3f2a685e"/>\n    <obo:OBI_0001937 rdf:datatype="http://www.w3.org/2001/XMLSchema#double">0.00018016897790311737</obo:OBI_0001937>\n  </rdf:Description>\n  <rdf:Des

In [7]:
# Process the machine readable statistical hypothesis test by completing the following SPARQL query that returns the p-value

q = g.query(
    """PREFIX obo: <http://purl.obolibrary.org/obo/>
    
    SELECT ?pvalue 
    WHERE {
        ?r a obo:STATO_0000304 .
        ?r obo:OBI_0000293 [ a obo:GO_0030350 ] .
        ?r obo:OBI_0000299 [ a obo:OBI_0000175 ] .
        ?s a obo:OBI_0000175 .
        ?s obo:OBI_0001938 [ a obo:OBI_0001931 ] .
        ?s obo:OBI_0001937 ?pvalue .
    }""")

for qs in q:
    print(qs[0])

0.00018016897790311737
